In [1]:
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
# !pip install torch-geometric

In [2]:
import numpy as np
import torch_geometric
from torch_geometric.datasets import Planetoid

In [3]:
use_cuda_if_available = False

# Load the dataset

In [4]:
dataset = Planetoid(root="tutorial1",name= "Cora")

##### Dataset properties

In [5]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of node features:\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)
print("number of the (first) graph's nodes:\t",dataset[0].num_nodes)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0
number of the (first) graph's nodes:	 2708


##### Dataset shapes

In [6]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [7]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("\n")
print("x:\t\t",dataset.data.x.shape)
print(dataset.data.x)
print("\n")
print("y:\t\t",dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [8]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [9]:
data = dataset[0]

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(in_channels=dataset.num_features,
                             out_channels=dataset.num_classes,
                             aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
device

device(type='cpu')

In [13]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [14]:
best_val_acc = test_acc = 0
cnt_early_stop = 0
for epoch in range(1,100):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    elif test_acc > 0:
        cnt_early_stop += 1
        if cnt_early_stop > 20:
            print("Early stop!")
            break
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch < 5 or epoch % 10 == 0:
        print(log.format(epoch, train_acc, best_val_acc, test_acc))
        if epoch == 4:
            print("...")

Epoch: 001, Train: 0.9929, Val: 0.5000, Test: 0.5160
Epoch: 002, Train: 1.0000, Val: 0.6280, Test: 0.6350
Epoch: 003, Train: 1.0000, Val: 0.6940, Test: 0.7010
Epoch: 004, Train: 1.0000, Val: 0.7180, Test: 0.7230
...
Epoch: 010, Train: 1.0000, Val: 0.7300, Test: 0.7190
Epoch: 020, Train: 1.0000, Val: 0.7300, Test: 0.7190
Early stop!


## Review


Q. Why valid and test accuracies are not increased as large as train accuracy?

This is because the ratio of train dataset is too small than others (valid, test datasets).

In [15]:
percent = np.array([sum(data.train_mask).item(), sum(data.val_mask).item(), sum(data.test_mask).item()]) / data.num_nodes * 100
print(f"Ratio of split[%]: train({percent[0]:.3f}) valid({percent[1]:.3f}) test({percent[2]:.3f})")

Ratio of split[%]: train(5.170) valid(18.464) test(36.928)


Only 5% of dataset (train) induces 70% of accuracy for val and test datasets.

The above results shows that inductive method is operated well. 